# Analyzing FEMA's National Flood Insurance Program (NFIP) Policies Data With DuckDB
Author: Mark Bauer

# OpenFEMA Dataset: FIMA NFIP Redacted Policies - v2
Federal Emergency Management Agency (FEMA), OpenFEMA Dataset: FIMA NFIP Redacted Policies - v2. Retrieved from https://www.fema.gov/openfema-data-page/fima-nfip-redacted-policies-v2. This product uses the FEMA OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about OpenFEMA's  [Terms and Conditions](https://www.fema.gov/about/openfema/terms-conditions).

**Dataset Description**:
>This dataset provides details on NFIP policy transactions. It is derived from the NFIP system of record, staged in the NFIP reporting platform and redacted to protect policy holder personally identifiable information.
>
>This dataset is not intended to be an official federal report, and should not be considered an official federal report.

**About the National Flood Insurance Program**:   
>Congress passed the National Flood Insurance Act (NFIA), 42 U.S.C. 4001 in 1968, creating the National Flood Insurance Program (NFIP) in order to reduce future flood losses through flood hazard identification, floodplain management, and providing insurance protection. The Department of Housing and Urban Development (HUD) originally administered the NFIP, and Congress subsequently transferred the NFIP to FEMA upon its creation in 1979. FEMA and insurance companies participating in FEMA's Write Your Own (WYO) program offer NFIP insurance coverage for building structures as well as for contents and personal property within the building structures to eligible and insurable properties. The WYO program began in 1983 with NFIP operating under Part B of the NFIA and allows FEMA to authorize private insurance companies to issue the Standard Flood Insurance Policy (SFIP) as FEMA's fiduciary and fiscal agent. FEMA administers NFIP by ensuring insurance applications are processed properly; determining correct premiums; renewing, reforming, and cancelling insurance policies; transferring policies from the seller of the property to the purchaser of the property in certain circumstances; and processing insurance claims.
>
>The paid premiums of SFIPs and claims payments for damaged property are processed through the National Flood Insurance Fund (NFIF). NFIF was established by the National Flood Insurance Act of 1968 (42 U.S.C. 4001, et seq.), and is a centralized premium revenue and fee-generated fund that supports NFIP, which holds these U.S. Treasury funds.
>
>The Flood Insurance Claims Manual (https://nfipservices.floodsmart.gov/insurance-manuals) provides claims guidance to WYOs, vendors, adjusters, and examiners so that policyholders experience consistent and reliable service. The Manual provides processes for handling claims from the notice of loss to final payment. The NFIP has provided answers to Frequently Asked Questions (FAQs) to assist the public in understanding and navigating the data our program makes available: https://www.fema.gov/sites/default/files/documents/fema_nfip-data-faqs.pdf.

**Data Dictionary**:  
View the data dictionary on OpenFEMA under the [Data Fields](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-policies-v2) section.

In [2]:
# import libraries
import duckdb
import pandas as pd

In [3]:
# reproducibility
%reload_ext watermark
%watermark -v -p duckdb,pandas

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

duckdb: 1.0.0
pandas: 1.5.1



In [4]:
# list datasets
%ls data/

FimaNfipClaims.parquet    FimaNfipPolicies.parquet


In [5]:
# preview size of file
!du -sh data/* | sort -rh

3.2G	data/FimaNfipPolicies.parquet
194M	data/FimaNfipClaims.parquet


# Redacted Policies: Before Getting Started
Please note that this dataset provides details on NFIP claims transactions and is ***redacted*** to protect policy holder personally identifiable information. The claim's `latitude` and `longitude` field **should not be used to represent the precise location of the insured building**. From the data dictionary:

>Latitude: **Approximate latitude of the insured building (to 1 decimal place)**. This represents the approximate location of the insured property. The precision has been lessened to ensure individual privacy. This may result in a point location that exists in an incorrect county or state. Use the state and county fields for record aggregation for these dimensions.
>
>Longitude: **Approximate longitude of the insured building (to 1 decimal place)**. This represents the approximate location of the insured property. The precision has been lessened to ensure individual privacy. This may result in a point location that exists in an incorrect county or state. Use the state and county fields for record aggregation for these dimensions.

For more information, visit the [Data Dictionary](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-policies-v2) and review [Frequently Asked Questions about NFIP Policies and Claims Data](https://nfipservices.floodsmart.gov/frequently-asked-questions-about-nfip-policies-and-claims-data).

This analysis presents all financial figures in nominal dollars. No adjustments for inflation have been made, meaning that the values reflect the actual dollar amounts at the time of measurement. Additionally, the date and time of data access can be found in the [download-data](https://github.com/mebauer/duckdb-fema-nfip/blob/main/download-data.ipynb) notebook.

# Create a DuckDB database instance using the Python client

In [6]:
# create a DuckDB database instance
con = duckdb.connect()

# create table claims of dataset
con.execute("""
    CREATE TABLE policies AS
        FROM read_parquet('data/FimaNfipPolicies.parquet')
""")

# sanity check
con.sql("""
    SELECT *
    FROM policies
    LIMIT 5
""").show()

┌──────────────────────┬────────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│ agricultureStructu…  │ baseFloodElevation │ … │   latitude   │  longitude   │          id          │
│       boolean        │    decimal(8,2)    │   │ decimal(9,1) │ decimal(9,1) │         uuid         │
├──────────────────────┼────────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│ false                │               NULL │ … │         32.2 │       -110.9 │ c3c498e0-39ee-4642…  │
│ false                │               NULL │ … │         37.6 │       -121.0 │ 6daee4b7-308b-453c…  │
│ false                │               NULL │ … │         35.4 │       -118.9 │ d4191676-0f6d-47bf…  │
│ false                │               NULL │ … │         39.0 │       -121.4 │ 9dac717a-9a1f-4323…  │
│ false                │               NULL │ … │         34.4 │       -119.6 │ e11197ee-65ef-4630…  │
├──────────────────────┴────────────────────┴───┴──────────────┴─────────

In [7]:
# list tables and schemas
con.sql("SHOW ALL TABLES").df()

,database,schema,name,column_names,column_types,temporary
0,memory,main,policies,"[agricultureStructureIndicator, baseFloodEleva...","[BOOLEAN, DECIMAL(8,2), SMALLINT, DATE, VARCHA...",False


In [23]:
# count of rows
con.sql("""
    SELECT COUNT(*) AS count_rows
    FROM policies
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│   69489458 │
└────────────┘

In [12]:
# count of columns
con.sql("""
    SELECT COUNT(column_name) AS count_columns
    FROM (DESCRIBE FROM policies)
""")

┌───────────────┐
│ count_columns │
│     int64     │
├───────────────┤
│            81 │
└───────────────┘

# Examine Dataset

## Column Info

In [13]:
# examine column datatypes
con.sql("""
    SELECT column_name, column_type
    FROM (DESCRIBE policies)
""").show(max_rows=80)

┌────────────────────────────────────────┬──────────────┐
│              column_name               │ column_type  │
│                varchar                 │   varchar    │
├────────────────────────────────────────┼──────────────┤
│ agricultureStructureIndicator          │ BOOLEAN      │
│ baseFloodElevation                     │ DECIMAL(8,2) │
│ basementEnclosureCrawlspaceType        │ SMALLINT     │
│ cancellationDateOfFloodPolicy          │ DATE         │
│ condominiumCoverageTypeCode            │ VARCHAR      │
│ construction                           │ BOOLEAN      │
│ crsClassCode                           │ SMALLINT     │
│ buildingDeductibleCode                 │ VARCHAR      │
│ contentsDeductibleCode                 │ VARCHAR      │
│ elevatedBuildingIndicator              │ BOOLEAN      │
│ elevationCertificateIndicator          │ VARCHAR      │
│ elevationDifference                    │ INTEGER      │
│ federalPolicyFee                       │ SMALLINT     │
│ ratedFloodZo

In [14]:
# approximate column null percentage
con.sql("""
    SELECT
        column_name,
        null_percentage
    FROM (SUMMARIZE FROM policies)
    ORDER BY null_percentage DESC
""").show(max_rows=80)

┌────────────────────────────────────────┬─────────────────┐
│              column_name               │ null_percentage │
│                varchar                 │  decimal(9,2)   │
├────────────────────────────────────────┼─────────────────┤
│ cancellationDateOfFloodPolicy          │           95.70 │
│ cancellationVoidanceReasonCode         │           95.70 │
│ obstructionType                        │           81.43 │
│ elevationCertificateIndicator          │           77.96 │
│ enclosureTypeCode                      │           74.92 │
│ insuranceToValueCode                   │           69.99 │
│ lowestAdjacentGrade                    │           68.84 │
│ lowestFloorElevation                   │           67.10 │
│ baseFloodElevation                     │           66.54 │
│ elevationDifference                    │           66.12 │
│ propertyPurchaseDate                   │           59.77 │
│ basementEnclosureCrawlspaceType        │           59.02 │
│ subsidizedRateType    

## Preview Data

In [15]:
# preview data
con.sql("""
    SELECT *
    FROM policies
    LIMIT 5
""").show()

┌──────────────────────┬────────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│ agricultureStructu…  │ baseFloodElevation │ … │   latitude   │  longitude   │          id          │
│       boolean        │    decimal(8,2)    │   │ decimal(9,1) │ decimal(9,1) │         uuid         │
├──────────────────────┼────────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│ false                │               NULL │ … │         32.2 │       -110.9 │ c3c498e0-39ee-4642…  │
│ false                │               NULL │ … │         37.6 │       -121.0 │ 6daee4b7-308b-453c…  │
│ false                │               NULL │ … │         35.4 │       -118.9 │ d4191676-0f6d-47bf…  │
│ false                │               NULL │ … │         39.0 │       -121.4 │ 9dac717a-9a1f-4323…  │
│ false                │               NULL │ … │         34.4 │       -119.6 │ e11197ee-65ef-4630…  │
├──────────────────────┴────────────────────┴───┴──────────────┴─────────

In [16]:
# preview data as pandas dataframe for readability
sql = """
    SELECT *
    FROM policies
    LIMIT 5
"""

# examine each column in sections because of large number of columns
con.sql(sql).df().iloc[:, :15]

,agricultureStructureIndicator,baseFloodElevation,basementEnclosureCrawlspaceType,cancellationDateOfFloodPolicy,condominiumCoverageTypeCode,construction,crsClassCode,buildingDeductibleCode,contentsDeductibleCode,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,federalPolicyFee,ratedFloodZone,hfiaaSurcharge
0,False,NaN,0,NaT,N,False,NaN,1,None,False,None,NaN,35,AE,0
1,False,NaN,0,NaT,N,False,NaN,5,None,False,None,NaN,35,AE,0
2,False,NaN,0,NaT,N,False,NaN,3,None,False,None,NaN,35,A11,0
3,False,NaN,0,NaT,N,False,NaN,0,0,True,None,NaN,13,A,0
4,False,NaN,0,NaT,N,False,NaN,1,0,True,None,NaN,35,AE,0


In [17]:
# slice through columns
con.sql(sql).df().iloc[:, 15:30]

,houseOfWorshipIndicator,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,nonProfitIndicator,numberOfFloorsInInsuredBuilding,obstructionType,occupancyType,originalConstructionDate,originalNBDate,policyCost,policyCount,policyEffectiveDate,policyTerminationDate,policyTermIndicator
0,False,NaN,NaN,NaN,False,1,NaN,1,1973-01-01,2006-04-26,541,1,2009-04-26,2010-04-26,1
1,False,NaN,NaN,NaN,False,1,NaN,1,2005-03-22,2022-01-05,515,1,2009-04-27,2010-04-27,1
2,False,NaN,NaN,NaN,False,1,NaN,1,1986-09-28,2005-05-24,1252,1,2009-05-24,2010-05-24,1
3,False,3.0,NaN,NaN,False,1,10.0,1,1992-07-01,2003-08-31,348,1,2009-08-31,2010-08-31,1
4,False,NaN,NaN,NaN,False,1,10.0,1,1960-01-01,2001-08-05,1251,1,2009-08-05,2010-08-05,1


In [18]:
# slice through columns
con.sql(sql).df().iloc[:, 30:45]

,postFIRMConstructionIndicator,primaryResidenceIndicator,rateMethod,regularEmergencyProgramIndicator,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,totalInsurancePremiumOfThePolicy,cancellationVoidanceReasonCode,subsidizedRateType,iccPremium,reserveFundAssessment,communityProbationSurcharge,premiumPaymentIndicator,buildingReplacementCost
0,False,True,1,R,False,83000,0,506,None,None,75,0,0,NaN,83000
1,True,False,1,R,False,165000,0,480,None,None,6,0,0,NaN,165000
2,False,True,1,R,False,250000,0,1217,None,None,60,0,0,NaN,250000
3,True,False,7,R,False,250000,100000,335,None,None,6,0,0,NaN,0
4,False,True,1,R,False,250000,0,1216,None,None,60,0,0,NaN,0


In [19]:
# slice through columns
con.sql(sql).df().iloc[:, 45:60]

,basicBuildingRate,additionalBuildingRate,basicContentsRate,AdditionalContentsRate,enclosureTypeCode,buildingDescriptionCode,insuranceToValueCode,postFirmVzoneIndicator,floodproofedIndicator,waitingPeriodType,rolloverTransferCode,endorsementEffectiveDate,propertyPurchaseDate,rentalPropertyIndicator,tenantIndicator
0,0.76,0.54,0.0,0.0,None,1,NaN,False,False,None,N,2009-04-26,NaT,False,False
1,0.78,0.21,0.0,0.0,N,1,NaN,False,False,None,N,2009-04-27,NaT,False,False
2,0.76,0.54,0.0,0.0,None,1,NaN,False,False,None,N,2009-05-24,NaT,False,False
3,0.00,0.00,0.0,0.0,None,1,NaN,False,False,None,R,2009-08-31,NaT,False,False
4,0.76,0.54,0.0,0.0,None,1,NaN,False,False,None,R,2009-08-05,NaT,False,False


In [24]:
# slice through columns
con.sql(sql).df().iloc[:, 60:75]

,stateOwnedIndicator,disasterAssistanceCoverageRequiredCode,mandatoryPurchaseFlag,grandfatheringTypeCode,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,programTypeIndicator,mapPanelNumber,mapPanelSuffix,floodZoneCurrent,femaRegion,propertyState,reportedCity,reportedZipCode
0,False,0,True,1,040076,0000,None,False,2282,L,AE,9,AZ,Currently Unavailable,85712
1,False,0,False,1,060384,0000,None,False,0532,F,None,9,CA,Currently Unavailable,95351
2,False,0,True,1,060075,0000,None,False,0745,B,None,9,CA,Currently Unavailable,93308
3,False,0,False,1,060460,0000,None,False,0045,D,None,9,CA,Currently Unavailable,95692
4,False,0,True,1,060331,0000,None,False,1392,H,AE,9,CA,Currently Unavailable,93108


In [25]:
# slice through columns
con.sql(sql).df().iloc[:, 75:]

,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,04019,04019003101,040190031011,32.2,-110.9,c3c498e0-39ee-4642-9537-bfd386347a70
1,06099,06099001603,060990016031,37.6,-121.0,6daee4b7-308b-453c-a1c3-6eab8dd90ab0
2,06029,06029005103,060290051031,35.4,-118.9,d4191676-0f6d-47bf-850c-08836f79cb58
3,06115,06115040800,061150408001,39.0,-121.4,9dac717a-9a1f-4323-8bb4-02e327e7a2ca
4,06083,06083001402,060830014022,34.4,-119.6,e11197ee-65ef-4630-a588-771637842dc8


In [26]:
# count duplicate IDs
con.sql("""
    SELECT
        id,
        COUNT(id) AS count
    FROM policies
    GROUP BY id
    HAVING count > 1
""").show()

┌──────┬───────┐
│  id  │ count │
│ uuid │ int64 │
├──────┴───────┤
│    0 rows    │
└──────────────┘



In [27]:
# earliest and latest record effective date
con.sql("""
    SELECT
        min(policyEffectiveDate) AS earliestPolicyEffectiveDate,
        max(policyEffectiveDate) AS earliestPolicyEffectiveDate
    FROM policies
""").show()

┌─────────────────────────────┬─────────────────────────────┐
│ earliestPolicyEffectiveDate │ earliestPolicyEffectiveDate │
│            date             │            date             │
├─────────────────────────────┼─────────────────────────────┤
│ 2009-01-01                  │ 2025-12-12                  │
└─────────────────────────────┴─────────────────────────────┘



In [28]:
# total insured units in dataset
con.sql("""
    SELECT SUM(policyCount) AS totalPolicyCount
    FROM policies
""").show()

┌──────────────────┐
│ totalPolicyCount │
│      int128      │
├──────────────────┤
│         85679492 │
└──────────────────┘



Policy Count:
>Insured units in an active status. A policy contract ceases to be in an active status as of the cancellation date or the expiration date. Residential Condominium Building Association Policy (RCBAP) contracts are stored as a single policy contract but insure multiple units and therefore represent multiple policies.

Source: https://www.fema.gov/openfema-data-page/fima-nfip-redacted-policies-v2

# Analysis
Note: All dollar amounts are reported in nominal terms and have not been adjusted for inflation.

## Policies Statistics

In [32]:
# policies by Effective Year
con.sql("""
    SELECT
        strftime('%Y', CAST(policyEffectiveDate AS TIMESTAMP))::BIGINT AS policyEffectiveYear,
        SUM(policyCount)::BIGINT AS policyCount,
        ROUND(SUM(totalInsurancePremiumOfThePolicy) / 1_000_000.0, 3) AS totalInsurancePremiumOfThePolicyM,
        ROUND(SUM(policyCost) / 1_000_000.0, 3) AS policyCostM,
        ROUND(SUM(totalBuildingInsuranceCoverage) / 1_000_000.0, 3) AS totalBuildingInsuranceCoverageM,
        ROUND(SUM(totalContentsInsuranceCoverage) / 1_000_000.0, 3) AS totalContentsInsuranceCoverageM,
        ROUND(SUM(totalBuildingInsuranceCoverage + totalContentsInsuranceCoverage) / 1_000_000.0, 3) AS totalInsuranceCoverageM
    FROM policies 
    WHERE policyEffectiveYear != '2025'
    GROUP BY policyEffectiveYear
    ORDER BY policyEffectiveYear DESC      
""").df()

,policyEffectiveYear,policyCount,totalInsurancePremiumOfThePolicyM,policyCostM,totalBuildingInsuranceCoverageM,totalContentsInsuranceCoverageM,totalInsuranceCoverageM
0,2024,4777399,4016.488,5255.495,1063821.946,247204.575,1311026.521
1,2023,4785555,3642.117,4821.280,1053495.710,253149.682,1306645.392
2,2022,4885565,3438.695,4588.656,1062285.568,261168.184,1323453.753
3,2021,5041304,3726.424,4931.348,1088926.656,278468.538,1367348.475
4,2020,5104136,3597.026,4769.859,1088157.530,281665.094,1369494.252
5,2019,5117681,3536.986,4630.337,1078533.525,277726.041,1356173.901
6,2018,5187350,3462.799,4571.601,1077662.414,275877.688,1353540.002
7,2017,5205874,3467.774,4601.696,1066041.689,268312.207,1334353.897
8,2016,5233261,3512.305,4673.950,1053072.607,256959.770,1310032.378
9,2015,5257480,3473.532,4441.183,1042901.090,250213.713,1293114.803


In [33]:
# policies by Flood Zone and Occupancy Type
con.sql("""
    SELECT
        propertyState,
        occupancyType,
        SUM(policyCount)::BIGINT AS policyCount,
        ROUND(SUM(totalInsurancePremiumOfThePolicy) / 1_000_000.0, 3) AS totalInsurancePremiumOfThePolicyM,
        ROUND(SUM(policyCost) / 1_000_000.0, 3) AS policyCostM,
        ROUND(SUM(totalBuildingInsuranceCoverage) / 1_000_000.0, 3) AS totalBuildingInsuranceCoverageM,
        ROUND(SUM(totalContentsInsuranceCoverage) / 1_000_000.0, 3) AS totalContentsInsuranceCoverageM,
        ROUND(SUM(totalBuildingInsuranceCoverage + totalContentsInsuranceCoverage) / 1_000_000.0, 3) AS totalInsuranceCoverageM
    FROM policies 
    GROUP BY propertyState, occupancyType
    ORDER BY policyCount DESC  
    LIMIT 20    
""").df()

,propertyState,occupancyType,policyCount,totalInsurancePremiumOfThePolicyM,policyCostM,totalBuildingInsuranceCoverageM,totalContentsInsuranceCoverageM,totalInsuranceCoverageM
0,FL,1,14462000,8406.257,10091.305,3031454.611,939763.076,3971143.802
1,FL,3,9494402,2275.875,2843.012,1633029.481,38529.641,1671547.335
2,TX,1,7967818,3959.361,4789.600,1565735.158,574688.524,2140390.477
3,LA,1,5486632,3255.772,3896.538,1012838.910,323302.374,1336122.756
4,FL,11,2807025,2607.998,3373.275,654079.863,190466.681,844546.544
5,CA,1,2686557,1826.871,2166.151,590655.118,147754.814,738356.684
6,FL,15,2157025,707.791,912.746,447009.416,775.810,447785.226
7,SC,1,1837538,1196.900,1454.756,407201.586,129281.866,536456.038
8,TX,11,1794518,1259.238,1660.684,399449.792,142893.525,542343.317
9,NJ,1,1735362,1705.030,2022.547,370510.296,73468.690,443954.097


In [34]:
# policies by Flood Zone and Occupancy Type
con.sql("""
    SELECT
        ratedFloodZone,
        occupancyType,
        SUM(policyCount)::BIGINT AS policyCount,
        ROUND(SUM(totalInsurancePremiumOfThePolicy) / 1_000_000.0, 3) AS totalInsurancePremiumOfThePolicyM,
        ROUND(SUM(policyCost) / 1_000_000.0, 3) AS policyCostM,
        ROUND(SUM(totalBuildingInsuranceCoverage) / 1_000_000.0, 3) AS totalBuildingInsuranceCoverageM,
        ROUND(SUM(totalContentsInsuranceCoverage) / 1_000_000.0, 3) AS totalContentsInsuranceCoverageM,
        ROUND(SUM(totalBuildingInsuranceCoverage + totalContentsInsuranceCoverage) / 1_000_000.0, 3) AS totalInsuranceCoverageM
    FROM policies 
    GROUP BY ratedFloodZone, occupancyType
    ORDER BY policyCount DESC  
    LIMIT 20    
""").df()

,ratedFloodZone,occupancyType,policyCount,totalInsurancePremiumOfThePolicyM,policyCostM,totalBuildingInsuranceCoverageM,totalContentsInsuranceCoverageM,totalInsuranceCoverageM
0,X,1,20305523,7527.790,9346.881,4227194.243,1665737.827,5892901.122
1,AE,1,16346901,14809.013,17697.133,3106612.066,645386.548,3751735.430
2,AE,3,7978763,2511.312,3117.882,1420222.617,21787.332,1441990.813
3,X,11,4283110,2461.291,3322.379,983850.425,377213.433,1361063.858
4,AE,11,3848741,4208.808,5421.344,829447.042,184646.289,1014093.331
5,X,3,3265196,778.514,1025.899,569014.051,28044.085,597048.126
6,AE,15,2112490,724.708,955.383,438906.202,713.195,439619.397
7,A,1,2037502,1859.142,2166.882,311501.508,42267.741,353732.979
8,AHB,1,1774357,625.753,752.182,372419.534,107852.104,480271.583
9,C,1,1656952,688.265,800.530,316148.018,120023.571,436165.667
